In [4]:
import os
import shutil
from PIL import Image
from IPython.display import display
import ipywidgets as widgets
from IPython.display import clear_output

# Directories (change these paths as needed)
image_folder = '../data/August24_CB01/non_dark_images/'  # Folder containing the images
flood_folder = '../data/August24_CB01/non_dark_images/flood'
noflood_folder = '../data/August24_CB01/non_dark_images/noflood'
unknown_folder = '../data/August24_CB01/non_dark_images/unknown'

# Create category directories if they don't exist
os.makedirs(flood_folder, exist_ok=True)
os.makedirs(noflood_folder, exist_ok=True)
os.makedirs(unknown_folder, exist_ok=True)

# Get the list of images and sort them by filename
images = sorted([f for f in os.listdir(image_folder) if f.endswith(('.png', '.jpg', '.jpeg'))])

# Widgets for buttons
flood_button = widgets.Button(description="Flood")
noflood_button = widgets.Button(description="No Flood")
unknown_button = widgets.Button(description="Unknown")

# Global variable to keep track of the current image index
current_image_index = 0

# Function to display image and buttons
def display_image(image_path, size=(600, 600)):  # Adjust the size tuple as needed
    img = Image.open(image_path)
    img_resized = img.resize(size)  # Resize the image to make it larger
    display(img_resized)

def sort_image(category):
    global current_image_index

    # Get current image
    image_file = images[current_image_index]
    image_path = os.path.join(image_folder, image_file)

    # Move the image to the appropriate folder
    if category == 'flood':
        shutil.move(image_path, os.path.join(flood_folder, image_file))
    elif category == 'noflood':
        shutil.move(image_path, os.path.join(noflood_folder, image_file))
    elif category == 'unknown':
        shutil.move(image_path, os.path.join(unknown_folder, image_file))

    # Move to the next image
    current_image_index += 1

    # Clear previous output and display the next image
    clear_output(wait=True)

    # Check if there are more images
    if current_image_index < len(images):
        display_image(os.path.join(image_folder, images[current_image_index]))
    else:
        print("No more images to sort!")

# Button click event handlers
def on_flood_button_clicked(b):
    sort_image('flood')

def on_noflood_button_clicked(b):
    sort_image('noflood')

def on_unknown_button_clicked(b):
    sort_image('unknown')

# Assign the button event handlers
flood_button.on_click(on_flood_button_clicked)
noflood_button.on_click(on_noflood_button_clicked)
unknown_button.on_click(on_unknown_button_clicked)

# Display the first image and buttons
if len(images) > 0:
    display_image(os.path.join(image_folder, images[current_image_index]))

    # Display the buttons
    display(widgets.HBox([flood_button, noflood_button, unknown_button]))
else:
    print("No images found in the folder.")

No more images to sort!


In [5]:
import pandas as pd

In [9]:
#attach to datalog
# Load the CSV file into a DataFrame
df = pd.read_csv('../data/August24_CB01/DataLog.txt', delimiter=',', names=['time', 'state', 'V'])
df.head()

,time,state,V
0,1722872887,NoFlood,8176
1,1722873247,NoFlood,8176
2,1722873607,NoFlood,8176
3,1722873967,NoFlood,8176
4,1722874327,NoFlood,8176


In [10]:
#Attach label to dataframe
# Function to label the dataframe based on the image filename
def label_images(df, folder, label):
    for filename in os.listdir(folder):
        if filename.endswith('.jpg'):
            # Extract time and state from filename (e.g., 1724789167NoFlood.jpg)
            base_name = filename.replace('.jpg', '')
            time_str = ''.join(filter(str.isdigit, base_name))  # Extract time digits
            state_str = ''.join(filter(str.isalpha, base_name))  # Extract state

            # Convert to integer for matching with the dataframe
            time_val = int(time_str)

            # Find the row in the dataframe that matches the time and state
            df.loc[(df['time'] == time_val) & (df['state'] == state_str), 'label'] = label

# Label the images in flood and noflood folders
label_images(df, flood_folder, 'Flood')
label_images(df, noflood_folder, 'NoFlood')

# Show the updated dataframe
df.head()

,time,state,V,label
0,1722872887,NoFlood,8176,NoFlood
1,1722873247,NoFlood,8176,NoFlood
2,1722873607,NoFlood,8176,NoFlood
3,1722873967,NoFlood,8176,NoFlood
4,1722874327,NoFlood,8176,NoFlood


In [11]:
df.to_csv('../data/August24_CB01/LabeledData.csv', index=False)